# api

> Main entrypoint api for markdown backend

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp markdown.api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *
from IPython.display import display, Markdown
import json
import os
from pathlib import Path
import tempfile

In [ ]:
#| hide
#| export
import io
from typing import Dict, Iterable, Sequence, Tuple

In [ ]:
# | export
from beetroot.api import handle_notebook

from beetroot.markdown.transformations import (
    EscapeEndLineSlashesWithinLatexMath,
    EscapeUnderscoresWithinLatexMath,
    MultiTransformer,
    ReplaceSingleDollarDelimiters,
    Transformer,
    Unindent,
)
from beetroot.markdown.source import (
    MarkdownSourceHandler,
)
from beetroot.markdown.outputs import MarkdownOutputHandler, MarkdownCompletion

In [ ]:
# | export
def export_markdown_notebook(
    nb_json: Dict,
    markdown_source_transformer=Transformer(),
    python_source_transformer=Transformer(),
    output_transformers_map: Dict[str, Transformer] = {},
) -> Tuple[str, Iterable[MarkdownCompletion]]:
    stream = io.StringIO()
    source_handler = MarkdownSourceHandler(
        stream=stream,
        markdown_source_transformer=markdown_source_transformer,
        python_source_transformer=python_source_transformer,
    )
    output_handler = MarkdownOutputHandler(stream, output_transformers_map)

    completions = handle_notebook(nb_json, source_handler, output_handler)

    stream.seek(0)
    return stream.read(), completions

In [ ]:
# Form the path to the testcase notebook
nbpath = os.path.dirname(os.path.realpath('__file__')) # Get path to this notebook
test_nb_path = Path(nbpath) / '../../test_artifacts' / 'testcase_notebook.ipynb'

# Load the testcase notebook as JSON
nb_json = json.loads(test_nb_path.read_text())

# Call export, get the text and the completion
md_text, completions = export_markdown_notebook(nb_json)

# Create a temporary directory we'll use for running the completion
with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)
    for completion in completions:
        completion(tmp_path)
        
    filename = None 
    with open(tmp_path / 'markdown.md', 'w') as md_file:
        md_file.write(md_text)
        filename = md_file.name 

    # Images won't render because their paths are relative to the markdown file,
    # not this notebook. That's unfortunate but I've decided to live with it. 
    # The solution would be to temporarily copy images from the output directory
    # to a local images/ directory here. But that seems brittle so I'm not doing
    # it for now. 
    display(Markdown(filename=filename))

```python
name = 'world'
print(f"hello, {name}")
```

```
hello, world
```

# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.

```python
# Multiline stream output
print("hello, world")
print("a second line")
print("a third line")
```

```
hello, world
a second line
a third line
```

```python
# Single line stream output with no trailing newline
import sys
sys.stdout.write("hello")
```

```
hello
```

```
5
```

```python
# Multiline stream output with no trailing newline
print("hello with a newline")
sys.stdout.write("hello")
```

```
hello with a newline
hello
```

```
5
```

```
This output should show but the source code shouldn't
```

```python
# The source code for this cell will show but its output won't 

print("This output should not show")
```

The cell below is intentionally left blank to test handling blank cells - do not remove it

```python

```

The above cell was intentionally left blank to test handling blank cells - do not remove it

```python
# Test latex output
from IPython.display import Latex

Latex('$\\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$')
```

$\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$

```python
# Test inline image
%matplotlib inline 
from matplotlib import pyplot as plt
from math import cos, radians, sin 

fig = plt.figure(figsize=(2, 1))
ax = fig.add_subplot(1, 1, 1)
ax.set(
    title="sin(x) and cos(x)"
)
xs = range(0,360)
ax.plot(xs, [sin(radians(x)) for x in xs], label="sin(x)")
ax.plot(xs, [cos(radians(x)) for x in xs], label="cos(x)")
_ = ax.legend()
```

![](images/302d164c-5d9f-48dc-b07e-2bdca82ba496.png)
```python

```



In [ ]:
# Form the path to the testcase notebook
nbpath = os.path.dirname(os.path.realpath('__file__'))  # Get path to this notebook
test_nb_path = (
    Path(nbpath) / '../../test_artifacts' / 'transformations_testcases.ipynb'
)

# Load the testcase notebook as JSON
nb_json = json.loads(test_nb_path.read_text())

# Setup transformers
transformer = MultiTransformer([
    EscapeUnderscoresWithinLatexMath(),
    EscapeEndLineSlashesWithinLatexMath(),
    ReplaceSingleDollarDelimiters()
])

# Export
md_text, _ = export_markdown_notebook(
    nb_json, 
    markdown_source_transformer=transformer,
    output_transformers_map={
        'text/markdown/data/output': transformer,
        'text/latex/data/output': transformer,
        'text/html/data/output': Unindent(),
    }
)

Markdown(md_text)

This is some markdown in which I will replace the dollar signs in the following inline expression: \\(a + b\\).

```python
from IPython.display import Markdown, Latex, YouTubeVideo
```

```python
Markdown('This is output markdown in which I will replace the dollar signs in the following inline expression: $a + b$.')
```

This is output markdown in which I will replace the dollar signs in the following inline expression: \\(a + b\\).

I do not want these block delimiters replaced: $$a + b$$

```python
Latex('$\\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$')
```

\\(\displaystyle w\_{11} x\_{11} + w\_{12} x\_{12} + w\_{21} x\_{21} + w\_{22} x\_{22}\\)

```python
# The html produced by this should be unindented
video_id = 'Y7eoO3hZQYY'
YouTubeVideo(video_id, allow_autoplay=True, loop=1, mute=1, playlist=video_id)
```

<iframe
    width="400"
    height="300"
    src="https://www.youtube.com/embed/Y7eoO3hZQYY?loop=1&mute=1&playlist=Y7eoO3hZQYY&autoplay=1"
    frameborder="0"
    allowfullscreen
    allow="autoplay"
></iframe>


```python
# Test fixing up of matrices:

text = ''.join([
    '$$\\begin{bmatrix}\n',
    '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
    '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
    '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
    '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
    '$$'        
])

Markdown(text)

```

$$\begin{bmatrix}
{x}\_{1,1} & {x}\_{1,2} & {x}\_{1,3} & {x}\_{1,4}\\\\
{x}\_{2,1} & {x}\_{2,2} & {x}\_{2,3} & {x}\_{2,4}\\\\
{x}\_{3,1} & {x}\_{3,2} & {x}\_{3,3} & {x}\_{3,4}\\\\
{x}\_{4,1} & {x}\_{4,2} & {x}\_{4,3} & {x}\_{4,4}\end{bmatrix}
$$



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()